In [ ]:
import os
import utils
import connectome_create
# viz_method = one of ['itkwidgets', 'vtk']
viz_method = 'vtk'
import json


# import some of our favorite packages
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import re
import warnings

%matplotlib inline
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import AgglomerativeClustering
import nglui.statebuilder as ngstbld

# # this is the EM specific package for querying the EM data
from caveclient import CAVEclient

# from meshparty import trimesh_io, trimesh_vtk
# from meshparty import skeletonize, skeleton_io, skeleton
import cloudvolume

%load_ext autoreload
%autoreload 2

In [ ]:
datastack_name = 'fanc_production_mar2021'

client = CAVEclient(datastack_name)

# Load Premotor table

In [ ]:
pre_to_df = connectome_create.load_pre_to_df(ext='ordered_no_frags')
pre_to_df.loc['motor'].index.to_frame().set_index('preferred_pool').to_csv('./dfs_saved/mn_to_pool_match.csv')

In [ ]:
pre_to_mn_df = connectome_create.load_pre_to_mn_df(ext='matched_typed_with_nt')

# Review that pre_to_df makes sense

In [ ]:
print(pre_to_df.shape)
print('input: {}'.format(pre_to_df.columns.names))
print('output: {}'.format(pre_to_df.index.names))
print(pre_to_df.columns.get_level_values('cell_class').unique())
print(pre_to_df.index.get_level_values('cell_class').unique())

pre_to_df.head()

# First, ensure that looking at the motor neurons gives the same connectome.

In [ ]:
All = slice(None)
input_tup = (All,All,All,All,All,All)
output_tup = ('motor',All,All,All,All,All)

pre_df_slice = pre_to_df.loc[output_tup,input_tup]

%config InlineBackend.figure_formats = ['png']
%matplotlib inline

cmap = utils.white_dense()

fig = plt.figure(1, figsize = [8,16])
ax = sns.heatmap(np.log10(pre_df_slice.T.to_numpy()+1), cmap=cmap)
cbar = ax.collections[0].colorbar
# here set the labelsize by 20
# cbar.ax.tick_params(labelsize=20)
cbar.set_label(label = 'log 10 # of synapses', size=24)
plt.xlabel('Motor Neurons', fontsize =18)
plt.ylabel('All to motor', fontsize =18)
plt.yticks(fontsize = 16)
plt.xticks(fontsize = 16)
plt.show()

# Not enough here. 

# Now plot hemilineage preference for MNs

In [ ]:
## Create lookuptable
cell_types = pre_to_mn_df.index.get_level_values('cell_type').unique().to_numpy()
nt_table = pd.read_csv('./annotations_hl/LacinHLTable.csv')
import re
pattern = re.compile("R[0-9]")

hl_lut = {'cell_type':cell_types,'hemilineage':cell_types}
hl_lut = pd.DataFrame(data=hl_lut)

for idx,r in hl_lut.iterrows():
    try:
        if np.isnan(r.hemilineage):
            continue
    except TypeError:
        pass
    hl = r.hemilineage

    if hl[0:2]=='RD':
        hl = '24B'
        # print('string was RD, now {}'.format(hl))

    if pattern.match(hl) or hl=='RVD' or hl=='Rcore_':
        # print('string was {}, now 8A'.format(hl))
        hl = '8A'

    if not hl.isalnum():
        hl = hl[0:-1]
        # print(hl)
    
    if not hl.isalnum():
        hl=hl[0:hl.find('_')]
        # print(hl)

    if hl == '9Ac':
        hl='9A'
        # print(hl)

    hl_lut.loc[idx,'hemilineage'] = hl

# merge with neurotransmitter table to get a lookuptable
hl_lut = hl_lut.merge(nt_table,how='outer',left_on='hemilineage',right_on='HL')
hl_lut = hl_lut.loc[~hl_lut.hemilineage.isna(),:]

hl_lut_reduced = hl_lut[['cell_type','NT','hemilineage']]

In [ ]:
%config InlineBackend.figure_formats = ['png']
%matplotlib inline

cmap = utils.white_dense()
def conn_heat_map(df,xticks,yticks,figsz=[16,16],savefig=False,figfilename=None,cmap=cmap):
    fig = plt.figure(1, figsize = figsz)
    ax = sns.heatmap(np.log10(df.T.to_numpy()+1), xticklabels=xticks, yticklabels=yticks, cmap=cmap)
    cbar = ax.collections[0].colorbar
    # here set the labelsize by 20
    # cbar.ax.tick_params(labelsize=20)
    cbar.set_label(label = 'log 10 # of synapses', size=24)
    plt.xlabel('Onto Output neurons', fontsize =18)
    plt.ylabel('From input neurons', fontsize =18)
    plt.yticks(fontsize = 16)
    plt.xticks(fontsize = 16)
    plt.show()
    if savefig:
        if figfilename is None:
            figfilename = 'flexor_pool_temp_' + (np.random.randint(10000,99999,size=None)).astype(str)
        fig.savefig('./figpanels/' + figfilename + '.svg',format='svg')

def lin_heat_map(df,xticks,yticks,figsz=[16,16],savefig=False,figfilename=None,cmap=cmap):
    fig = plt.figure(1, figsize = figsz)
    ax = sns.heatmap(df.T.to_numpy(), xticklabels=xticks, yticklabels=yticks, cmap=cmap)
    cbar = ax.collections[0].colorbar
    # here set the labelsize by 20
    # cbar.ax.tick_params(labelsize=20)
    cbar.set_label(label = 'linear', size=24)
    plt.xlabel('Onto Output neurons', fontsize =18)
    plt.ylabel('From input neurons', fontsize =18)
    plt.yticks(fontsize = 16)
    plt.xticks(fontsize = 16)
    plt.show()
    if figfilename is not None:
        fig.savefig('./figpanels/' + figfilename + '.svg',format='svg')

In [ ]:
# simplify the hemilineages
input_tup = ('local',All,All,All,All,All)
output_tup = ('motor',All,All,All,All,All)

local_onto_motor_df = pre_to_df.loc[output_tup,input_tup]

# just operate on the index, where the nt will be
pmn_index_df = local_onto_motor_df.columns.to_frame().reset_index(drop=True)
pmn_index_df = pmn_index_df.join(hl_lut_reduced.set_index('cell_type'),how='left',on='cell_type',lsuffix='_x',rsuffix='_y')
pmn_index_df

# reorder the matrix
pmn_index_df = pmn_index_df[['preferred_pool','NT_x','classification_system','cell_type','hemilineage','segID']]
pmn_index_df.hemilineage.value_counts()


local_onto_motor_df.columns = pd.MultiIndex.from_frame(pmn_index_df)
local_onto_motor_df

hl_onto_motor_df = local_onto_motor_df.groupby(by='hemilineage',axis=1).sum()
hl_onto_pool_df = hl_onto_motor_df.groupby(by='preferred_pool',axis=0).sum()
hl_onto_pool_df
pool_keys = [
        'thorax_swing',
        'thorax_stance',
        'trochanter_extension',
        'trochanter_flexion',
        'femur_reductor',
        'tibia_extensor',
        'main_tibia_flexor',
        # 'auxiliary_tibia_flexor_A',
        'auxiliary_tibia_flexor_B',
        'auxiliary_tibia_flexor_E',
        'ltm',
        'tarsus_depressor_med_venU',
        'tarsus_depressor_noid',
        ]

hl_onto_pool_df = hl_onto_pool_df.loc[pool_keys,:]

xticks=hl_onto_pool_df.index.get_level_values('preferred_pool')
yticks=hl_onto_pool_df.columns.get_level_values('hemilineage')

conn_heat_map(df=hl_onto_pool_df, xticks=xticks,yticks=yticks,figsz=[4,16])

hl_onto_pool_df = (hl_onto_pool_df.T/hl_onto_pool_df.T.sum(axis=0)).T

lin_heat_map(df=hl_onto_pool_df, xticks=xticks,yticks=yticks,figsz=[4,4],figfilename='HL_contacts_from_local')



In [ ]:
# simplify the hemilineages
input_tup = ('local',All,All,All,All,All)
output_tup = ('motor',All,All,All,All,All)

local_onto_motor_df = pre_to_df.loc[output_tup,input_tup]

# # just operate on the index, where the nt will be
# pmn_index_df = local_onto_motor_df.columns.to_frame().reset_index(drop=True)
# pmn_index_df = pmn_index_df.join(hl_lut_reduced.set_index('cell_type'),how='left',on='cell_type',lsuffix='_x',rsuffix='_y')
# pmn_index_df

# # reorder the matrix
# pmn_index_df = pmn_index_df[['preferred_pool','NT_x','classification_system','cell_type','hemilineage','segID']]
# pmn_index_df.hemilineage.value_counts()


# local_onto_motor_df.columns = pd.MultiIndex.from_frame(pmn_index_df)
# local_onto_motor_df

nt_onto_motor_df = local_onto_motor_df.groupby(by='NT',axis=1).sum()
nt_onto_pool_df = nt_onto_motor_df.groupby(by='preferred_pool',axis=0).sum()
nt_onto_pool_df
pool_keys = [
        'thorax_swing',
        'thorax_stance',
        'trochanter_extension',
        'trochanter_flexion',
        'femur_reductor',
        'tibia_extensor',
        'main_tibia_flexor',
        # 'auxiliary_tibia_flexor_A',
        'auxiliary_tibia_flexor_B',
        'auxiliary_tibia_flexor_E',
        'ltm',
        'tarsus_depressor_med_venU',
        'tarsus_depressor_noid',
        ]

nt_onto_pool_df = nt_onto_pool_df.loc[pool_keys,:]

# Normalize
nt_onto_pool_df = (nt_onto_pool_df.T/nt_onto_pool_df.T.sum(axis=0)).T
# nt_onto_pool_df = nt_onto_pool_df/nt_onto_pool_df.sum(axis=0)
# nt_onto_pool_df = nt_onto_pool_df.T

xticks=nt_onto_pool_df.index.get_level_values('preferred_pool')
yticks=nt_onto_pool_df.columns.get_level_values('NT')

lin_heat_map(df=nt_onto_pool_df, xticks=xticks,yticks=yticks,figsz=[4,1],figfilename='NT_contacts_from_local')


# Now look at the small slice of local premotor neurons that prefer flexors, onto everything

In [ ]:
All = slice(None)
# pool_keys = [
        # 'thorax_swing',
        # 'thorax_stance',
        # 'trochanter_extension',
        # 'trochanter_flexion',
        # 'femur_reductor',
        # 'tibia_extensor',
        # 'main_tibia_flexor',
        # # 'auxiliary_tibia_flexor_A',
        # 'auxiliary_tibia_flexor_B',
        # 'auxiliary_tibia_flexor_E',
        # 'ltm',
        # 'tarsus_depressor_med_venU',
        # 'tarsus_depressor_noid',
        # ]

# input: ['cell_class', 'preferred_pool', 'NT', 'classification_system', 'cell_type', 'segID']
# output: ['cell_class', 'preferred_pool', 'NT', 'classification_system', 'cell_type', 'post_pt_root_id']

In [ ]:
# How many intersegmental neurons? # ~350
# segID	motor	has_soma	sensory	 neck	local	

def get_motory_fraction(pre_df,c_tup,compress_other_classes=False):
    ct_df = pre_df.loc[:,c_tup] # a more generalized version of the above

    # pd.Series

    # pd.DataFrame of fractional inputs
    cell_classes = ct_df.index.get_level_values('cell_class').unique()
    frac_df = pd.DataFrame(index=cell_classes,columns=ct_df.columns)

    for key in cell_classes:
        frac_df.loc[key,:] = ct_df.loc[key,:].sum(axis=0)

    
    if compress_other_classes:
        otherlist = ['non_t1_motor', 
                    'descending_post', 
                    'sensory_post',
                    'ascending_post', 
                    'neurmodulatory_ascending_neuron',
                    'vnc_non_premotor']
        frac_df.loc['other',:] = frac_df.loc[otherlist,:].sum(axis=0)
        
    ct_to_all    = ct_df.loc[All,:].sum(axis=0)                         # calculate the total
    frac_df = frac_df/ct_to_all                                         # divide by the total
    x = [i for i in range(len(ct_df.loc['motor',:].sum(axis=0)))]   
    frac_df.loc['x',:] = x                                              # this x is the original order of the cell_class
    frac_df.loc['total',:] = ct_to_all
    frac_df = frac_df.sort_values(['motor'],axis=1,ascending=False)    # sort according to how t1 motory

    sorted_total=frac_df.loc['total',:].to_numpy()                      # keep the sorted total 
    frac_df  = frac_df.drop('total',axis=0)                             # drop the total row
    
    if compress_other_classes:
        frac_df = frac_df.drop(otherlist,axis=0)
        frac_df = frac_df.sort_values(['other'],axis=1,ascending=False)    # sort according to how t1 motory
    
    return frac_df, sorted_total                                       


## (Not currently sorting by motoryness) First get the motoryness of the Tibia extensor, flexor pool

In [ ]:
# first sort by synapses onto the pool
input_tup = ('local',['tibia_extensor','main_tibia_flexor'],All,All,All,All,All)
frac_df,sorted_total = get_motory_fraction(pre_to_df,input_tup,compress_other_classes=False)

pre_onto_ti_ext_flex_df = pre_to_df.loc[:,input_tup]
# pre_onto_ti_ext_flex_df = pre_onto_ti_ext_flex_df.loc[:,frac_df.columns]
# pre_onto_ti_ext_flex_df

# # # locs, labels = plt.xticks(ticks=x, labels=lbls, rotation=90)
#           #'motor', 'nont1',    'desc',     'sens',     'ascend',   'local',    'intr',     'vnc',      'fragments'
# colors = ["#910951","#CC4893",  "#79D0F7",  "#93E5B6",  "#6084CC",  "#7D688E",  "#CC9FCC",  '#5C085E',   '#cccccc']
# cmap = sns.set_palette(sns.color_palette(colors))
# x = [i for i in range(len(sorted_total))]
# ax = frac_df.T.plot(x='x',kind='bar', stacked=True, legend = True,width=1,cmap=cmap,figsize=(16,40)) 

## Show the Motor neurons first

In [ ]:
# input_tup = ('local',['tibia_extensor','main_tibia_flexor'],All,All,All,All,All)
output_tup = ('motor',['tibia_extensor','main_tibia_flexor'],All,All,All,All,All)

pre_onto_ti_mns_df = pre_onto_ti_ext_flex_df.loc[output_tup,:]
# weird neurons
print('no NT: {}'.format(pre_onto_ti_mns_df.loc[:,~pre_onto_ti_mns_df.columns.get_level_values('NT').isin(['Ach','GABA','Glu'])].columns.get_level_values('segID').to_list()))
# Get rid of the last two pMNs
pre_onto_ti_mns_df = pre_onto_ti_mns_df.loc[:,pre_onto_ti_mns_df.columns.get_level_values('NT').isin(['Ach','GABA','Glu'])]

# xticklabels = pre_onto_ti_ext_flex_df.index.get_level_values('preferred_pool')
# yticklabels = pre_df_ti_ext_flex.columns.get_level_values('NT')

# conn_heat_map(df=pre_df_ti_ext_flex, xticks=xticklabels,yticks=yticklabels,figsz=[4,16])

# order by NT
cols_ntsorted,c_array = pre_onto_ti_mns_df.columns.sortlevel(level='NT',ascending=True,sort_remaining=False)
mn_nt_sorted = pre_onto_ti_mns_df.loc[:,cols_ntsorted]

# order by pool
cols_ntsorted,c_array = mn_nt_sorted.columns.sortlevel(level='preferred_pool',ascending=True,sort_remaining=False)
output_tup = ('local',['tibia_extensor','main_tibia_flexor'],All,All,All,All,All)
mn_nt_pool_sorted = mn_nt_sorted.loc[:,output_tup]

mnpoollabel = mn_nt_pool_sorted.index.get_level_values('preferred_pool')
yticklabels = mn_nt_pool_sorted.columns.get_level_values('NT')

conn_heat_map(df=mn_nt_pool_sorted, xticks=mnpoollabel,yticks=yticklabels,figsz=[4,16])

## Order local neurons by NT, and pool

In [ ]:
# input_tup = ('local',['tibia_extensor','main_tibia_flexor'],All,All,All,All)
output_tup = ('local',['tibia_extensor','main_tibia_flexor'],All,All,All,All,All)

pre_onto_ti_local_df = pre_onto_ti_ext_flex_df.loc[output_tup,:]

# then sort index by motoryness
# pre_onto_ti_local_df = pre_onto_ti_local_df.loc[frac_df.columns,:]


In [ ]:

# weird neurons
print('no NT: {}'.format(pre_onto_ti_local_df.loc[:,~pre_onto_ti_local_df.columns.get_level_values('NT').isin(['Ach','GABA','Glu'])].columns.get_level_values('segID').to_list()))
# Get rid of the last two pMNs
pre_onto_ti_local_df = pre_onto_ti_local_df.loc[pre_onto_ti_local_df.index.get_level_values('NT').isin(['Ach','GABA','Glu']),pre_onto_ti_local_df.columns.get_level_values('NT').isin(['Ach','GABA','Glu'])]

# sort by 
rows_ntsorted,array = pre_onto_ti_local_df.index.sortlevel(level='NT',ascending=True,sort_remaining=False)
cols_ntsorted,c_array = pre_onto_ti_local_df.columns.sortlevel(level='NT',ascending=True,sort_remaining=False)
local_nt_sorted = pre_onto_ti_local_df.loc[rows_ntsorted,cols_ntsorted]

# order by pool
output_tup = ('local',['tibia_extensor','main_tibia_flexor'],All,All,All,All,All)
local_nt_pool_sorted = local_nt_sorted.loc[output_tup,output_tup]

# mnpoollabel = local_nt_pool_sorted.index.get_level_values('preferred_pool')
yticklabels = local_nt_pool_sorted.columns.get_level_values('NT')

xpool = local_nt_pool_sorted.index.get_level_values('preferred_pool')
pool_dict = {'tibia_extensor':'ext','main_tibia_flexor':'flex'}
xpool = [pool_dict[key] for key in xpool]

xNT = local_nt_pool_sorted.index.get_level_values('NT').to_list()
xpoolnt = [x + "_" + y for x,y in zip(xpool,xNT)]

conn_heat_map(df=local_nt_pool_sorted, xticks=xpoolnt,yticks=xpoolnt,figsz=[16,16])

In [ ]:
# Format a nicer figure with both plots side by side
# do it in different colors too, then cut out different versions in illustrator
fig, ax = plt.subplots(1, 2, figsize=(32,16),gridspec_kw={'width_ratios': [1, 3.5]})

cmap = utils.white_dense()
# cmap = sns.color_palette("light:seagreen", as_cmap=True)

# def conn_heat_map(df,xticks,yticks,figsz=[16,16],savefig=False,figfilename=None,cmap=cmap):
sns.heatmap(np.log10(mn_nt_pool_sorted.T.to_numpy()+1),ax=ax[0], xticklabels=mnpoollabel, yticklabels=xpoolnt, cmap=cmap,square=True,cbar=False,vmax=np.log10(633),vmin=0)
plt.xlabel('Onto MNs', fontsize =18)
plt.ylabel('From input neurons', fontsize =18)
sns.heatmap(np.log10(local_nt_pool_sorted.T.to_numpy()+1),ax=ax[1], xticklabels=xpoolnt, yticklabels=xpoolnt,square=True, cmap=cmap,vmax=np.log10(633),vmin=0)
plt.xlabel('Onto local pmns', fontsize =18)

cbar = ax[1].collections[0].colorbar
cbar.set_label(label = 'log 10 # of synapses', size=24)
# plt.yticks(fontsize = 16)
# plt.xticks(fontsize = 16)
# plt.show()

fig.savefig('./figpanels/' + 'local_ti_ext_flex_by_nt' + '.svg',format='svg')


# Pick out some interesting neurons
Plot the inputs and output, synapses?

## 1) parallel excitation

In [ ]:
def build_ach_synapses_onto_pools(render_neurons,syn_df_dict,clr_dict): # #F6921E

    img_layer = ngstbld.ImageLayerConfig(name='fanc_v4',
                                 source=client.info.image_source(),
                                 )
    seg_layer = ngstbld.SegmentationLayerConfig(name = 'seg',
                                        source =  client.info.segmentation_source(),
                                        fixed_ids = render_neurons)

    sb1 = ngstbld.StateBuilder(layers=[img_layer, seg_layer], client=client, resolution=[4.3,4.3,45])
    state_builders = [sb1]

    df_list = [None]

    for key in syn_df_dict.keys():
        print('layer {}'.format(key))
        pool_points = ngstbld.PointMapper(point_column='pre_pt_position') ######################## change this to toggle rendering of pre- or post- synaptic points
        pool_layer = ngstbld.AnnotationLayerConfig(name=key,
                                        mapping_rules=pool_points,
                                        linked_segmentation_layer=seg_layer.name,
                                        color=clr_dict[key] )
        sb_pool = ngstbld.StateBuilder(layers=[pool_layer],client=client)
        state_builders.append(sb_pool)
        df_list.append(syn_df_dict[key])

    sb=ngstbld.ChainedStateBuilder(statebuilders=state_builders)
    # state = sb.render_state(data_list=[ach_df,gaba_df,glu_df],return_as='dict')

    print('{} sbs, {} dfs'.format(len(state_builders),len(df_list)))

    state = sb.render_state(data_list=df_list,return_as='dict')
    state_id = client.state.upload_state_json(state)

    DEFAULT_NGL = "https://neuromancer-seung-import.appspot.com/"

    url = client.state.build_neuroglancer_url(state_id, ngl_url=DEFAULT_NGL)

    return url

In [ ]:
timestamp = connectome_create.get_timestamp()

extn_ach_pmn = mn_nt_pool_sorted.loc[:,('local','tibia_extensor','Ach',All,All)]
extn_ach_pmn = extn_ach_pmn.iloc[:,[0]].columns.get_level_values('segID').to_list()[0]  # A 3A neuron to flexor pool
extn_ach_pmn

flex_ach_pmn = mn_nt_pool_sorted.loc[:,('local','main_tibia_flexor','Ach',All,All)]
flex_ach_pmn = flex_ach_pmn.iloc[:,[2]].columns.get_level_values('segID').to_list()[0]  # A 3A neuron to flexor pool
flex_ach_pmn

render_neurons = [flex_ach_pmn, extn_ach_pmn]

FETI_segid = mn_nt_pool_sorted.loc[(All,'tibia_extensor',All,All,"feti",All)].index.get_level_values('post_pt_root_id').to_list()
FETi_input_from_extn_ach_pmn = client.materialize.synapse_query(post_ids = FETI_segid, pre_ids=extn_ach_pmn, timestamp=timestamp)

SETI_segid = mn_nt_pool_sorted.loc[(All,'tibia_extensor',All,All,"seti",All)].index.get_level_values('post_pt_root_id').to_list()
SETi_input_from_extn_ach_pmn = client.materialize.synapse_query(post_ids = SETI_segid, pre_ids=extn_ach_pmn, timestamp=timestamp)

syn_df_dict = {'FETi':FETi_input_from_extn_ach_pmn,'SETi':SETi_input_from_extn_ach_pmn}
clr_dict = {'FETi': '#D38327','SETi':'#FEE39F'}

flexor_rnk = mn_nt_pool_sorted.loc[(All,'main_tibia_flexor',All,'main_tibia_flexor',All,All)].index.get_level_values('cell_type').to_list()
flexor_rnk[1] = '1'
flexor_id = mn_nt_pool_sorted.loc[(All,'main_tibia_flexor',All,'main_tibia_flexor',All,All)].index.get_level_values('post_pt_root_id').to_list()

clr_list = ['#0000AA','#2222AA','#4444CC','#6666FF','#AAAAFF']
for rnk,id,clr in zip(flexor_rnk,flexor_id,clr_list):
    flexor_segid = mn_nt_pool_sorted.loc[(All,'main_tibia_flexor',All,All,All,id)].index.get_level_values('post_pt_root_id').to_list()
    syn_df = client.materialize.synapse_query(post_ids = flexor_segid, pre_ids=flex_ach_pmn, timestamp=timestamp)
    syn_df_dict[rnk] = syn_df
    clr_dict[rnk] = clr

build_ach_synapses_onto_pools(render_neurons,syn_df_dict,clr_dict)

## parallel inhibition

In [ ]:
def build_ach_synapses_onto_pools(render_neurons,syn_df_dict,clr_dict): # #F6921E

    img_layer = ngstbld.ImageLayerConfig(name='fanc_v4',
                                 source=client.info.image_source(),
                                 )
    seg_layer = ngstbld.SegmentationLayerConfig(name = 'seg',
                                        source =  client.info.segmentation_source(),
                                        fixed_ids = render_neurons)

    sb1 = ngstbld.StateBuilder(layers=[img_layer, seg_layer], client=client, resolution=[4.3,4.3,45])
    state_builders = [sb1]

    df_list = [None]

    for key in syn_df_dict.keys():
        print('layer {}'.format(key))
        pool_points = ngstbld.PointMapper(point_column='pre_pt_position') ######################## change this to toggle rendering of pre- or post- synaptic points
        pool_layer = ngstbld.AnnotationLayerConfig(name=key,
                                        mapping_rules=pool_points,
                                        linked_segmentation_layer=seg_layer.name,
                                        color=clr_dict[key] )
        sb_pool = ngstbld.StateBuilder(layers=[pool_layer],client=client)
        state_builders.append(sb_pool)
        df_list.append(syn_df_dict[key])

    sb=ngstbld.ChainedStateBuilder(statebuilders=state_builders)
    # state = sb.render_state(data_list=[ach_df,gaba_df,glu_df],return_as='dict')

    print('{} sbs, {} dfs'.format(len(state_builders),len(df_list)))

    state = sb.render_state(data_list=df_list,return_as='dict')
    state_id = client.state.upload_state_json(state)

    DEFAULT_NGL = "https://neuromancer-seung-import.appspot.com/"

    url = client.state.build_neuroglancer_url(state_id, ngl_url=DEFAULT_NGL)

    return url

In [ ]:
def build_sphere_synapses_onto(render_neurons,syn_df_dict,clr_dict): # #F6921E

    img_layer = ngstbld.ImageLayerConfig(name='fanc_v4',
                                 source=client.info.image_source(),
                                 )
    seg_layer = ngstbld.SegmentationLayerConfig(name = 'seg',
                                        source =  client.info.segmentation_source(),
                                        fixed_ids = render_neurons)

    sb1 = ngstbld.StateBuilder(layers=[img_layer, seg_layer], client=client, resolution=[4.3,4.3,45])
    state_builders = [sb1]

    df_list = [None]

    for key in syn_df_dict.keys():
        print('layer {}'.format(key))
        # pool_points = ngstbld.PointMapper(point_column='pre_pt_position') ######################## change this to toggle rendering of pre- or post- synaptic points
        pool_spheres = ngstbld.SphereMapper(center_column='pre_pt_position', radius_column='radius')
        pool_layer = ngstbld.AnnotationLayerConfig(name=key,
                                        mapping_rules=pool_spheres,
                                        linked_segmentation_layer=seg_layer.name,
                                        color=clr_dict[key] )
        sb_pool = ngstbld.StateBuilder(layers=[pool_layer],client=client)
        state_builders.append(sb_pool)
        df_list.append(syn_df_dict[key])

    sb=ngstbld.ChainedStateBuilder(statebuilders=state_builders)
    # state = sb.render_state(data_list=[ach_df,gaba_df,glu_df],return_as='dict')

    print('{} sbs, {} dfs'.format(len(state_builders),len(df_list)))

    state = sb.render_state(data_list=df_list,return_as='dict')
    state_id = client.state.upload_state_json(state)

    DEFAULT_NGL = "https://neuromancer-seung-import.appspot.com/"

    url = client.state.build_neuroglancer_url(state_id, ngl_url=DEFAULT_NGL)

    return url

In [ ]:
timestamp = connectome_create.get_timestamp()

flex_glu_pmn = mn_nt_pool_sorted.loc[:,('local','main_tibia_flexor','Glu',All,All)].iloc[:,[2]] # 21A, 21A, R1
flex_glu_pmn_rt_id = flex_glu_pmn.columns.get_level_values('segID').to_list()[0]

extn_ach_pmn = mn_nt_pool_sorted.loc[:,('local','tibia_extensor','Ach',All,All)].iloc[:,[2]] # 0 is 3A, 1 is 1A, 2 is 3A
# flex_ach_pmn = flex_ach_pmn.iloc[:,[2]].columns.get_level_values('segID').to_list()[0]  # A 3A neuron to flexor pool
extn_ach_pmn_rt_id = extn_ach_pmn.columns.get_level_values('segID').to_list()[0]

render_neurons = [  extn_ach_pmn_rt_id, 
                    flex_glu_pmn_rt_id]

flex_glu_pmn


FETI_segid = mn_nt_pool_sorted.loc[(All,'tibia_extensor',All,All,"feti",All)].index.get_level_values('post_pt_root_id').to_list()
FETi_input_from_extn_ach_pmn = client.materialize.synapse_query(post_ids = FETI_segid, pre_ids=extn_ach_pmn_rt_id, timestamp=timestamp)

SETI_segid = mn_nt_pool_sorted.loc[(All,'tibia_extensor',All,All,"seti",All)].index.get_level_values('post_pt_root_id').to_list()
SETi_input_from_extn_ach_pmn = client.materialize.synapse_query(post_ids = SETI_segid, pre_ids=extn_ach_pmn_rt_id, timestamp=timestamp)

syn_df_dict = {'FETi':FETi_input_from_extn_ach_pmn,'SETi':SETi_input_from_extn_ach_pmn}
clr_dict = {'FETi': '#D38327','SETi':'#FEE39F'}

flexor_rnk = mn_nt_pool_sorted.loc[(All,'main_tibia_flexor',All,'main_tibia_flexor',All,All)].index.get_level_values('cell_type').to_list()
flexor_rnk[1] = '1'
flexor_id = mn_nt_pool_sorted.loc[(All,'main_tibia_flexor',All,'main_tibia_flexor',All,All)].index.get_level_values('post_pt_root_id').to_list()

clr_list = ['#0000AA','#2222AA','#4444CC','#6666FF','#AAAAFF']
for rnk,id,clr in zip(flexor_rnk,flexor_id,clr_list):
    flexor_segid = mn_nt_pool_sorted.loc[(All,'main_tibia_flexor',All,All,All,id),:].index.get_level_values('post_pt_root_id').to_list()
    syn_df = client.materialize.synapse_query(post_ids = flexor_segid, pre_ids=flex_glu_pmn_rt_id, timestamp=timestamp)
    syn_df.loc[:,'radius'] = int(100)
    syn_df_dict[rnk] = syn_df
    clr_dict[rnk] = clr

build_ach_synapses_onto_pools(render_neurons,syn_df_dict,clr_dict)
#build_sphere_synapses_onto(render_neurons,syn_df_dict,clr_dict)

In [ ]:
mn_nt_pool_sorted.loc[:,('local','tibia_extensor','Ach',All,All)].iloc[:,[2,3,4]]

In [ ]:
mn_nt_pool_sorted.loc[:,('local','main_tibia_flexor','Ach',All,All)].iloc[:,0:4] 

In [ ]:
mn_nt_pool_sorted.loc[(All,'main_tibia_flexor',All,All,All,id),:]

In [ ]:
utils.save_df_csv(pre_to_df.loc['motor'].index.to_frame(),'motor_pool_segids')

## 2) reciprocal inhibition

In [ ]:
def build_ach_synapses_onto_glu(render_neurons,syn_df_dict,clr_dict): # #F6921E

    img_layer = ngstbld.ImageLayerConfig(name='fanc_v4',
                                 source=client.info.image_source(),
                                 )
    seg_layer = ngstbld.SegmentationLayerConfig(name = 'seg',
                                        source =  client.info.segmentation_source(),
                                        fixed_ids = render_neurons)

    sb1 = ngstbld.StateBuilder(layers=[img_layer, seg_layer], client=client, resolution=[4.3,4.3,45])
    state_builders = [sb1]

    df_list = [None]

    for key in syn_df_dict.keys():
        print('layer {}'.format(key))
        pool_points = ngstbld.PointMapper(point_column='pre_pt_position') ######################## change this to toggle rendering of pre- or post- synaptic points
        pool_layer = ngstbld.AnnotationLayerConfig(name=key,
                                        mapping_rules=pool_points,
                                        linked_segmentation_layer=seg_layer.name,
                                        color=clr_dict[key] )
        sb_pool = ngstbld.StateBuilder(layers=[pool_layer],client=client)
        state_builders.append(sb_pool)
        df_list.append(syn_df_dict[key])

    sb=ngstbld.ChainedStateBuilder(statebuilders=state_builders)

    print('{} sbs, {} dfs'.format(len(state_builders),len(df_list)))

    state = sb.render_state(data_list=df_list,return_as='dict')
    state_id = client.state.upload_state_json(state)

    DEFAULT_NGL = "https://neuromancer-seung-import.appspot.com/"

    url = client.state.build_neuroglancer_url(state_id, ngl_url=DEFAULT_NGL)

    return url

### Extension preferring Glu neurons

In [ ]:
timestamp = connectome_create.get_timestamp()

# Two glutamatergic neurons participate in this circuit. Plotting both, with their connections in different layers

flex_ach_onto_ext_glu_pmn = local_nt_pool_sorted.loc[('local','tibia_extensor','Glu',All,All),('local','main_tibia_flexor','Ach',All,All)]
flex_ach_onto_ext_glu_pmn = flex_ach_onto_ext_glu_pmn.iloc[[0,2],:]
render_neurons = flex_ach_onto_ext_glu_pmn.index.get_level_values('post_pt_root_id').to_list()
syn_df_dict = {}
clr_dict = {}
for n in render_neurons:
    glu_pmn = flex_ach_onto_ext_glu_pmn.loc[(All,All,All,All,All,n)]
    ach_pmns = glu_pmn.columns.get_level_values('segID').to_list()
    ach_onto_glu_df = client.materialize.synapse_query(post_ids = n, pre_ids=ach_pmns, timestamp=timestamp)

    key = 'Ach_' + glu_pmn.index.get_level_values('cell_type').to_list()[0]
    syn_df_dict[key] = ach_onto_glu_df
    clr_dict[key]  = '#15B01A'

for n in render_neurons:
    ext_glu_onto_ext_mns = mn_nt_pool_sorted.loc[('motor','tibia_extensor',All,All,All),('local','tibia_extensor','Glu',All,All,n)]
    mns = ext_glu_onto_ext_mns.index.get_level_values('post_pt_root_id').to_list()
    glu_onto_ext_df = client.materialize.synapse_query(pre_ids = n, post_ids=mns, timestamp=timestamp)

    key = 'Out_' + ext_glu_onto_ext_mns.columns.get_level_values('cell_type').to_list()[0]
    syn_df_dict[key] = glu_onto_ext_df
    clr_dict[key]  = '#F6921E'

build_ach_synapses_onto_pools(render_neurons,syn_df_dict,clr_dict)

### Flexion preferring Glu neurons

In [ ]:
# Two glutamatergic neurons participate in this circuit. Plotting both, with their connections in different layers

extn_ach_onto_flex_glu_pmn = local_nt_pool_sorted.loc[('local','main_tibia_flexor','Glu',All,All),('local','tibia_extensor','Ach',All,All)]
extn_ach_onto_flex_glu_pmn = extn_ach_onto_flex_glu_pmn.iloc[[3],:] # Note: 0 is also 21A, seems cool that these two neurons mediate different motifs
extn_ach_onto_flex_glu_pmn

render_neurons = extn_ach_onto_flex_glu_pmn.index.get_level_values('post_pt_root_id').to_list()
syn_df_dict = {}
clr_dict = {}
for n in render_neurons:
    glu_pmn = extn_ach_onto_flex_glu_pmn.loc[(All,All,All,All,All,n)]
    ach_pmns = glu_pmn.columns.get_level_values('segID').to_list()
    ach_onto_glu_df = client.materialize.synapse_query(post_ids = n, pre_ids=ach_pmns, timestamp=timestamp)

    key = 'Ach_' + glu_pmn.index.get_level_values('cell_type').to_list()[0]
    syn_df_dict[key] = ach_onto_glu_df
    clr_dict[key]  = '#15B01A'

for n in render_neurons:
    flex_glu_onto_flex_mns = mn_nt_pool_sorted.loc[('motor','main_tibia_flexor',All,All,All),('local','main_tibia_flexor','Glu',All,All,n)]
    mns = flex_glu_onto_flex_mns.index.get_level_values('post_pt_root_id').to_list()
    glu_onto_flex_df = client.materialize.synapse_query(pre_ids = n, post_ids=mns, timestamp=timestamp)

    key = 'Out_' + flex_glu_onto_flex_mns.columns.get_level_values('cell_type').to_list()[0]
    syn_df_dict[key] = glu_onto_flex_df
    clr_dict[key]  = '#2E3191'

build_ach_synapses_onto_pools(render_neurons,syn_df_dict,clr_dict)

## 3) disinhibition

### Extension preferring Glu neurons

In [ ]:
timestamp = connectome_create.get_timestamp()

# Two glutamatergic neurons participate in this circuit. Plotting both, with their connections in different layers

flex_glu_onto_ext_glu_pmn = local_nt_pool_sorted.loc[('local','tibia_extensor','Glu',All,All),('local','main_tibia_flexor','Glu',All,All)]
flex_glu_onto_ext_glu_pmn = flex_glu_onto_ext_glu_pmn.iloc[[0,2],:]
render_neurons = flex_glu_onto_ext_glu_pmn.index.get_level_values('post_pt_root_id').to_list()
print(render_neurons)
syn_df_dict = {}
clr_dict = {}
for n in render_neurons:
    glu_pmn = flex_glu_onto_ext_glu_pmn.loc[(All,All,All,All,All,n)]
    ach_pmns = glu_pmn.columns.get_level_values('segID').to_list()
    ach_onto_glu_df = client.materialize.synapse_query(post_ids = n, pre_ids=ach_pmns, timestamp=timestamp)

    key = 'Ach_' + glu_pmn.index.get_level_values('cell_type').to_list()[0]
    syn_df_dict[key] = ach_onto_glu_df
    clr_dict[key]  = '#FEDF08'

for n in render_neurons:
    ext_glu_onto_ext_mns = mn_nt_pool_sorted.loc[('motor','tibia_extensor',All,All,All),('local','tibia_extensor','Glu',All,All,n)]
    mns = ext_glu_onto_ext_mns.index.get_level_values('post_pt_root_id').to_list()
    glu_onto_ext_df = client.materialize.synapse_query(pre_ids = n, post_ids=mns, timestamp=timestamp)

    key = 'Out_' + ext_glu_onto_ext_mns.columns.get_level_values('cell_type').to_list()[0]
    syn_df_dict[key] = glu_onto_ext_df
    clr_dict[key]  = '#F6921E'

build_ach_synapses_onto_pools(render_neurons,syn_df_dict,clr_dict)

### Flexion preferring GABA neuron

In [ ]:
# Two glutamatergic neurons participate in this circuit. Plotting both, with their connections in different layers

extn_glu_onto_flex_gaba_pmn = local_nt_pool_sorted.loc[('local','main_tibia_flexor','GABA',All,All),('local','tibia_extensor','Glu',All,All)]
extn_glu_onto_flex_gaba_pmn = extn_glu_onto_flex_gaba_pmn.iloc[[0],:] # Note: 0 is also 21A, seems cool that these two neurons mediate different motifs

render_neurons = extn_glu_onto_flex_gaba_pmn.index.get_level_values('post_pt_root_id').to_list()
print(render_neurons)
syn_df_dict = {}
clr_dict = {}
for n in render_neurons:
    gaba_pmn = extn_glu_onto_flex_gaba_pmn.loc[(All,All,All,All,All,n)]
    glu_pmns = gaba_pmn.columns.get_level_values('segID').to_list()
    glu_onto_gaba_df = client.materialize.synapse_query(post_ids = n, pre_ids=glu_pmns, timestamp=timestamp)

    key = 'Glu_' + gaba_pmn.index.get_level_values('cell_type').to_list()[0]
    syn_df_dict[key] = glu_onto_gaba_df
    clr_dict[key]  = '#FEDF08'

for n in render_neurons:
    flex_gaba_onto_flex_mns = mn_nt_pool_sorted.loc[('motor','main_tibia_flexor',All,All,All),('local','main_tibia_flexor','GABA',All,All,n)]
    mns = flex_gaba_onto_flex_mns.index.get_level_values('post_pt_root_id').to_list()
    gaba_onto_flex_df = client.materialize.synapse_query(pre_ids = n, post_ids=mns, timestamp=timestamp)

    key = 'Out_' + flex_gaba_onto_flex_mns.columns.get_level_values('cell_type').to_list()[0]
    syn_df_dict[key] = gaba_onto_flex_df
    clr_dict[key]  = '#2E3191'

build_ach_synapses_onto_pools(render_neurons,syn_df_dict,clr_dict)

## 4) Divergent presynpatic inhibition

In [ ]:
timestamp = connectome_create.get_timestamp()

flex_gaba_onto_flex_ach_pmn = local_nt_pool_sorted.loc[('local','main_tibia_flexor','Ach',All,All),('local','main_tibia_flexor','GABA',All,All)]
flex_gaba_onto_flex_ach_pmn = flex_gaba_onto_flex_ach_pmn.iloc[:,[3]] # [0,2,3]]
print(flex_gaba_onto_flex_ach_pmn.head())
render_neurons = flex_gaba_onto_flex_ach_pmn.columns.get_level_values('segID').to_list()
render_neurons

ach_pmns = flex_gaba_onto_flex_ach_pmn.index.get_level_values('post_pt_root_id').to_list()
gaba_onto_ach_df = client.materialize.synapse_query(pre_ids = render_neurons, post_ids=ach_pmns, timestamp=timestamp)

syn_df_dict = {'gaba_onto_ach':gaba_onto_ach_df}
clr_dict = {'gaba_onto_ach': '#CB416B'}

flex_gaba_onto_flex_mns = mn_nt_pool_sorted.loc[('motor','main_tibia_flexor',All,'main_tibia_flexor',All),('local','main_tibia_flexor','GABA',All,All)]

mns = flex_gaba_onto_flex_mns.index.get_level_values('post_pt_root_id').to_list()
gaba_onto_flex_df = client.materialize.synapse_query(pre_ids = render_neurons, post_ids=mns, timestamp=timestamp)

syn_df_dict['MN']=gaba_onto_flex_df
clr_dict['MN']='#2E3191'

build_ach_synapses_onto_pools(render_neurons,syn_df_dict,clr_dict)

## 5) Convergent input onto big 21A neuron

In [ ]:
# See https://github.com/seung-lab/NeuroglancerAnnotationUI/blob/master/src/nglui/statebuilder/helpers.py#L207 for details

def build_synapses(render_neurons=[],ach_df=None,gaba_df=None,glu_df=None,motor_df=None,motor_clr='#2E3191'): # #F6921E

    img_layer = ngstbld.ImageLayerConfig(name='fanc_v4',
                                 source=client.info.image_source(),
                                 )
    seg_layer = ngstbld.SegmentationLayerConfig(name = 'seg',
                                        source =  client.info.segmentation_source(),
                                        fixed_ids = render_neurons)

    sb1 = ngstbld.StateBuilder(layers=[img_layer, seg_layer], client=client, resolution=[4.3,4.3,45])
    state_builders = [sb1]

    df_list = [None]

    if ach_df is not None:
        ach_points = ngstbld.PointMapper(point_column='pre_pt_position') ######################## change this to toggle rendering of pre- or post- synaptic points
        ach_layer = ngstbld.AnnotationLayerConfig(name='Ach',
                                           mapping_rules=ach_points,
                                           linked_segmentation_layer=seg_layer.name,
                                           color='#15B01A' )
        sb_ach = ngstbld.StateBuilder(layers=[ach_layer],client=client)
        state_builders.append(sb_ach)
        df_list.append(ach_df)

    if gaba_df is not None:
        gaba_points = ngstbld.PointMapper(point_column='pre_pt_position') ######################## change this to toggle rendering of pre- or post- synaptic points
        gaba_layer = ngstbld.AnnotationLayerConfig(name='GABA',
                                        mapping_rules=ach_points,
                                        linked_segmentation_layer=seg_layer.name,
                                        color='#CB416B' )
        sb_gaba = ngstbld.StateBuilder(layers=[gaba_layer],client=client)
        state_builders.append(sb_gaba)
        df_list.append(gaba_df)

    if glu_df is not None:
        glu_points = ngstbld.PointMapper(point_column='pre_pt_position') ######################## change this to toggle rendering of pre- or post- synaptic points
        glu_layer = ngstbld.AnnotationLayerConfig(name='Glu',
                                        mapping_rules=ach_points,
                                        linked_segmentation_layer=seg_layer.name,
                                        color='#FEDF08' )
        sb_glu = ngstbld.StateBuilder(layers=[glu_layer],client=client)
        state_builders.append(sb_glu)
        df_list.append(glu_df)

    if motor_df is not None:
        mn_points = ngstbld.PointMapper(point_column='pre_pt_position') ######################## change this to toggle rendering of pre- or post- synaptic points
        mn_layer = ngstbld.AnnotationLayerConfig(name='MN',
                                        mapping_rules=ach_points,
                                        linked_segmentation_layer=seg_layer.name,
                                        color=motor_clr )
        sb_mn = ngstbld.StateBuilder(layers=[mn_layer],client=client)
        state_builders.append(sb_mn)
        df_list.append(motor_df)

    sb=ngstbld.ChainedStateBuilder(statebuilders=state_builders)
    # state = sb.render_state(data_list=[ach_df,gaba_df,glu_df],return_as='dict')
    state = sb.render_state(data_list=df_list,return_as='dict')
    state_id = client.state.upload_state_json(state)

    DEFAULT_NGL = "https://neuromancer-seung-import.appspot.com/"

    url = client.state.build_neuroglancer_url(state_id, ngl_url=DEFAULT_NGL)

    return url




In [ ]:
timestamp = connectome_create.get_timestamp()

# 5) Convergent input from Glu neurons onto big 21A neuron
glu_pmn = local_nt_pool_sorted.loc[('local','main_tibia_flexor','Glu',All,All),:].iloc[[0],:]
onto_list = glu_pmn.index.get_level_values('post_pt_root_id').to_list()

ach_pmns = glu_pmn.loc[('local','main_tibia_flexor','Glu',All,All),('local','main_tibia_flexor','Ach',All,All)]
from_list = ach_pmns.columns.get_level_values('segID').to_list()
ach_syn_df = client.materialize.synapse_query(post_ids = onto_list, pre_ids=from_list, timestamp=timestamp) # Takes list

gaba_pmns = glu_pmn.loc[:,('local','main_tibia_flexor','GABA',All,All)]
from_list = gaba_pmns.columns.get_level_values('segID').to_list()
gaba_syn_df = client.materialize.synapse_query(post_ids = onto_list, pre_ids=from_list, timestamp=timestamp) # Takes list

glu_pmns = glu_pmn.loc[:,('local','main_tibia_flexor','Glu',All,All)]
from_list = glu_pmns.columns.get_level_values('segID').to_list()
glu_syn_df = client.materialize.synapse_query(post_ids = onto_list, pre_ids=from_list, timestamp=timestamp) # Takes list

onto_mns = mn_nt_pool_sorted.index.get_level_values('post_pt_root_id').to_list()
mn_df = client.materialize.synapse_query(post_ids = onto_mns, pre_ids=onto_list, timestamp=timestamp) # Takes list

%matplotlib inline
build_synapses(onto_list[0],ach_syn_df,gaba_syn_df,glu_syn_df,mn_df)
